## Importing Libraries

In [1]:
import pandas as pd
import optuna

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool, cv
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error, r2_score

## Configurations

In [2]:
# path of training dataset
data_path = 'data/processed/train_processed.csv'

FOLDS=5 # no. of folds for cross validation
SPLIT=0.2 # splitting size
SEED=42 # seed for reproducibility

In [3]:
# loading the training dataset
df = pd.read_csv(data_path)

In [4]:
# separating dependent and independent variables
X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

In [5]:
# splitting the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=SPLIT, random_state=SEED)

## Model Training

In [6]:
# models for training
models = {
    'LinearRegression': LinearRegression(),
    'Lasso Regression': Lasso(),
    'Ridge Regression': Ridge(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'XGBRegressor': XGBRegressor(),
    'CatBoostRegressor': CatBoostRegressor(verbose=False),
    'LGBMRegressor': LGBMRegressor(verbose=-1),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'Support Vector Regressor': SVR(),
    
}

In [7]:
# comparing models
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    results.append({
        'Model': name,
        'MSE': mean_squared_error(y_val, y_pred),
        'MAE': mean_absolute_error(y_val, y_pred),
        'RMSE': root_mean_squared_error(y_val, y_pred),
        'R2 Score': r2_score(y_val, y_pred)
    })

    results_df = pd.DataFrame(results).sort_values('R2 Score', ascending=False)

In [8]:
results_df

,Model,MSE,MAE,RMSE,R2 Score
7,CatBoostRegressor,0.043007,0.158435,0.207382,0.658159
8,LGBMRegressor,0.043259,0.159167,0.207989,0.656155
6,XGBRegressor,0.043972,0.159065,0.209695,0.650490
4,RandomForestRegressor,0.047394,0.164718,0.217703,0.623288
5,GradientBoostingRegressor,0.052574,0.178736,0.229290,0.582119
9,KNeighborsRegressor,0.064605,0.194755,0.254174,0.486494
10,Support Vector Regressor,0.073277,0.209302,0.270697,0.417562
3,DecisionTreeRegressor,0.080345,0.210005,0.283452,0.361382
2,Ridge Regression,0.087789,0.234355,0.296292,0.302215
0,LinearRegression,0.087790,0.234351,0.296293,0.302209


## Tuning Hyper-Parameters using Optuna

### 1. CatBoostRegressor

In [9]:
# objective function for tuning CatBoostRegressor
def objective(trail):
    params = {
        'iterations': trail.suggest_int('iterations', 500, 1500),
        'learning_rate': trail.suggest_float('learning_rate', 0.01, 0.1),
        'depth': trail.suggest_int('depth', 1, 10),
        'l2_leaf_reg': trail.suggest_int('l2_leaf_reg', 1, 10),
        'border_count': trail.suggest_int('border_count',10, 150),
        'random_strength': trail.suggest_float('random_strength', 0.1, 1.0),
        'bagging_temperature': trail.suggest_float('bagging_temperature', 0, 1.0),

        'loss_function': 'RMSE',
        'eval_metric': 'R2',
        'random_seed': SEED,
        'verbose': False
    }

    cv_results = cv(
        pool=Pool(X_train, y_train),
        params=params,
        fold_count=FOLDS,
        shuffle=True,
        stratified=False,
        verbose=False
    )

    best_r2 = max(cv_results['test-R2-mean'])

    return best_r2

In [10]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print(f'Best Parameters: {study.best_params}')

[I 2026-02-15 11:34:38,748] A new study created in memory with name: no-name-2ad8992c-c96c-48b0-9f39-f3997366b69d


Training on fold [0/5]

bestTest = 0.6004312836
bestIteration = 705

Training on fold [1/5]

bestTest = 0.6015121855
bestIteration = 705

Training on fold [2/5]

bestTest = 0.6023627135
bestIteration = 705

Training on fold [3/5]

bestTest = 0.5953916581
bestIteration = 703

Training on fold [4/5]


[I 2026-02-15 11:34:47,127] Trial 0 finished with value: 0.6000378087016994 and parameters: {'iterations': 706, 'learning_rate': 0.08355212565823263, 'depth': 2, 'l2_leaf_reg': 4, 'border_count': 88, 'random_strength': 0.17058882949803897, 'bagging_temperature': 0.8274903777181077}. Best is trial 0 with value: 0.6000378087016994.



bestTest = 0.6005113759
bestIteration = 705

Training on fold [0/5]

bestTest = 0.6067770299
bestIteration = 808

Training on fold [1/5]

bestTest = 0.6047470232
bestIteration = 809

Training on fold [2/5]

bestTest = 0.6038939759
bestIteration = 809

Training on fold [3/5]

bestTest = 0.5982549207
bestIteration = 809

Training on fold [4/5]


[I 2026-02-15 11:34:56,908] Trial 1 finished with value: 0.6039236479032108 and parameters: {'iterations': 810, 'learning_rate': 0.08378983578163723, 'depth': 2, 'l2_leaf_reg': 9, 'border_count': 90, 'random_strength': 0.5812927009815783, 'bagging_temperature': 0.17361401475828553}. Best is trial 1 with value: 0.6039236479032108.


...


bestTest = 0.6538492136
bestIteration = 871

Training on fold [0/5]

bestTest = 0.6716783955
bestIteration = 1094

Training on fold [1/5]

bestTest = 0.656858526
bestIteration = 750

Training on fold [2/5]

bestTest = 0.6588873811
bestIteration = 838

Training on fold [3/5]

bestTest = 0.6508008618
bestIteration = 979

Training on fold [4/5]


[I 2026-02-15 12:05:16,006] Trial 48 finished with value: 0.6579033067412469 and parameters: {'iterations': 1411, 'learning_rate': 0.0883580796238213, 'depth': 8, 'l2_leaf_reg': 4, 'border_count': 117, 'random_strength': 0.2594815985764553, 'bagging_temperature': 0.3399957364621039}. Best is trial 46 with value: 0.6629254143641886.



bestTest = 0.6524429986
bestIteration = 925

Training on fold [0/5]

bestTest = 0.6735100798
bestIteration = 1494

Training on fold [1/5]

bestTest = 0.658374774
bestIteration = 1435

Training on fold [2/5]

bestTest = 0.6611061125
bestIteration = 1418

Training on fold [3/5]

bestTest = 0.6539901142
bestIteration = 1461

Training on fold [4/5]


[I 2026-02-15 12:05:44,510] Trial 49 finished with value: 0.6600580216945401 and parameters: {'iterations': 1499, 'learning_rate': 0.07022894120370848, 'depth': 6, 'l2_leaf_reg': 3, 'border_count': 131, 'random_strength': 0.23306091290608671, 'bagging_temperature': 0.5048570214358719}. Best is trial 46 with value: 0.6629254143641886.



bestTest = 0.6536218569
bestIteration = 1495

Best Parameters: {'iterations': 1410, 'learning_rate': 0.07260960744718559, 'depth': 8, 'l2_leaf_reg': 5, 'border_count': 150, 'random_strength': 0.33427443535082535, 'bagging_temperature': 0.26724639014963236}


In [11]:
# validating the tuned CatBoostRegressor
best_params = study.best_params

tuned_CBR = CatBoostRegressor(**best_params)
tuned_CBR.fit(X_train, y_train)
y_pred = tuned_CBR.predict(X_val)

catboost_results = {
    'MSE': mean_squared_error(y_val, y_pred),
    'MAE': mean_absolute_error(y_val, y_pred),
    'RMSE': root_mean_squared_error(y_val, y_pred),
    'R2 Score': r2_score(y_val, y_pred)
}

catboost_results_df = pd.DataFrame([catboost_results])

0:	learn: 0.3464500	total: 7.43ms	remaining: 10.5s
1:	learn: 0.3365121	total: 12.6ms	remaining: 8.85s
2:	learn: 0.3279915	total: 17.5ms	remaining: 8.19s
3:	learn: 0.3202199	total: 22ms	remaining: 7.73s
4:	learn: 0.3130131	total: 27ms	remaining: 7.58s
5:	learn: 0.3068145	total: 32.6ms	remaining: 7.62s
6:	learn: 0.3008564	total: 39.3ms	remaining: 7.88s
7:	learn: 0.2957078	total: 44.3ms	remaining: 7.77s
8:	learn: 0.2912627	total: 49.2ms	remaining: 7.66s
9:	learn: 0.2869100	total: 53.5ms	remaining: 7.49s
10:	learn: 0.2830463	total: 58.3ms	remaining: 7.42s
11:	learn: 0.2796712	total: 62.9ms	remaining: 7.33s
12:	learn: 0.2771517	total: 67.2ms	remaining: 7.22s
13:	learn: 0.2736100	total: 72ms	remaining: 7.18s
14:	learn: 0.2714338	total: 76.6ms	remaining: 7.12s
15:	learn: 0.2685333	total: 81.3ms	remaining: 7.08s
16:	learn: 0.2659293	total: 85.6ms	remaining: 7.02s
17:	learn: 0.2640862	total: 91ms	remaining: 7.03s
18:	learn: 0.2618646	total: 96.1ms	remaining: 7.03s
19:	learn: 0.2604424	total: 10

In [12]:
catboost_results_df

,MSE,MAE,RMSE,R2 Score
0,0.043379,0.158078,0.208276,0.655207


### 2. XGBRegressor

In [13]:
# objective function for tuning XGBRegressor
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample',0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 2.5),
        'random_state': SEED
    }

    model = XGBRegressor(**params)
    score = cross_val_score(model, X_train, y_train, cv=FOLDS, scoring='r2').mean()

    return score

In [14]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print(f'Best Parameters: {study.best_params}')

[I 2026-02-15 12:05:51,607] A new study created in memory with name: no-name-c0d4f35c-5896-4813-97df-d1e12874639a
[I 2026-02-15 12:05:54,174] Trial 0 finished with value: 0.5110075285674286 and parameters: {'n_estimators': 807, 'learning_rate': 0.10692436086264254, 'max_depth': 1, 'min_child_weight': 7, 'subsample': 0.501512144201497, 'colsample_bytree': 0.9440698392204363, 'gamma': 0.028065658946798244, 'reg_alpha': 0.7286171157667828, 'reg_lambda': 1.791030699972128}. Best is trial 0 with value: 0.5110075285674286.
[I 2026-02-15 12:05:55,170] Trial 1 finished with value: 0.5985862704200715 and parameters: {'n_estimators': 239, 'learning_rate': 0.05679423579932837, 'max_depth': 4, 'min_child_weight': 10, 'subsample': 0.7719813661481965, 'colsample_bytree': 0.586388427230838, 'gamma': 0.3226254392872, 'reg_alpha': 0.013210019232570258, 'reg_lambda': 2.2094235607860107}. Best is trial 1 with value: 0.5985862704200715.
[I 2026-02-15 12:05:59,225] Trial 2 finished with value: 0.6361320306

Best Parameters: {'n_estimators': 242, 'learning_rate': 0.04117835121824566, 'max_depth': 9, 'min_child_weight': 9, 'subsample': 0.7579805271544652, 'colsample_bytree': 0.9653026653268035, 'gamma': 0.047590182336904444, 'reg_alpha': 0.3285697976953739, 'reg_lambda': 2.3084809413552914}


In [15]:
# validating the tuned XGBRegressor
best_params = study.best_params

tuned_XGB = XGBRegressor(**best_params)
tuned_XGB.fit(X_train, y_train)
y_pred = tuned_XGB.predict(X_val)

xgb_results = {
    'MSE': mean_squared_error(y_val, y_pred),
    'MAE': mean_absolute_error(y_val, y_pred),
    'RMSE': root_mean_squared_error(y_val, y_pred),
    'R2 Score': r2_score(y_val, y_pred)
}

xgb_results_df = pd.DataFrame([xgb_results])

In [16]:
xgb_results_df

,MSE,MAE,RMSE,R2 Score
0,0.042351,0.156845,0.205794,0.663374


### 3. LGBMRegressor

In [17]:
# objective function for LGBMRegressor

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'num_leaves': trial.suggest_int('num_leaves', 31, 200),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 200),
        'subsample': trial.suggest_float('subsample', 0.8, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.8, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1.0),
        'random_state': SEED,
        'verbose':-1
    }

    model = LGBMRegressor(**params)
    score = cross_val_score(model, X_train, y_train, cv=FOLDS, scoring='r2').mean()

    return score

In [18]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print(f'Best Parameters: {study.best_params}')

[I 2026-02-15 12:07:10,991] A new study created in memory with name: no-name-31a787f7-8850-492c-bcb0-cd9e23aa42fb
[I 2026-02-15 12:07:13,280] Trial 0 finished with value: 0.6631691144745019 and parameters: {'n_estimators': 885, 'learning_rate': 0.05793190116634715, 'max_depth': 5, 'num_leaves': 71, 'min_child_samples': 91, 'subsample': 0.9652059832209393, 'colsample_bytree': 0.8985673009588011, 'reg_alpha': 0.3404548685806681, 'reg_lambda': 0.49507360853372706}. Best is trial 0 with value: 0.6631691144745019.
[I 2026-02-15 12:07:14,081] Trial 1 finished with value: 0.6544768789046049 and parameters: {'n_estimators': 261, 'learning_rate': 0.13125973502479962, 'max_depth': 6, 'num_leaves': 134, 'min_child_samples': 195, 'subsample': 0.8570093594384277, 'colsample_bytree': 0.811079951880712, 'reg_alpha': 0.3044009144404005, 'reg_lambda': 0.11514807344251721}. Best is trial 0 with value: 0.6631691144745019.
[I 2026-02-15 12:07:19,186] Trial 2 finished with value: 0.653417726523158 and para

Best Parameters: {'n_estimators': 432, 'learning_rate': 0.08217083200300074, 'max_depth': 9, 'num_leaves': 56, 'min_child_samples': 69, 'subsample': 0.8652292202872996, 'colsample_bytree': 0.9747385858191636, 'reg_alpha': 0.5977255765124226, 'reg_lambda': 0.22391749616360299}


In [19]:
# validating the tuned LGBMRegressor
best_params = study.best_params

tuned_LGBM = LGBMRegressor(**best_params)
tuned_LGBM.fit(X_train, y_train)
y_pred = tuned_LGBM.predict(X_val)

lgbm_results = {
    'MSE': mean_squared_error(y_val, y_pred),
    'MAE': mean_absolute_error(y_val, y_pred),
    'RMSE': root_mean_squared_error(y_val, y_pred),
    'R2 Score': r2_score(y_val, y_pred)
}

lgbm_results_df = pd.DataFrame([lgbm_results])

In [20]:
lgbm_results_df

,MSE,MAE,RMSE,R2 Score
0,0.042204,0.155888,0.205436,0.664544


### 4. Ensemble of tuned models

In [21]:
# ensemble of tuned models
ensemble = StackingRegressor(
    estimators=[
        ('catboost', tuned_CBR),
        ('lgbm', tuned_LGBM),
        ('xgb', tuned_XGB)
], final_estimator=Ridge(), cv=FOLDS)

In [22]:
# training the ensemble on training dataset
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_val)

ensemble_results = {
    'MSE': mean_squared_error(y_val, y_pred),
    'MAE': mean_absolute_error(y_val, y_pred),
    'RMSE': root_mean_squared_error(y_val, y_pred),
    'R2 Score': r2_score(y_val, y_pred)
}

ensemble_results_df = pd.DataFrame([ensemble_results])

0:	learn: 0.3464500	total: 5.96ms	remaining: 8.4s
1:	learn: 0.3365121	total: 11.8ms	remaining: 8.29s
2:	learn: 0.3279915	total: 16.8ms	remaining: 7.87s
3:	learn: 0.3202199	total: 22ms	remaining: 7.73s
4:	learn: 0.3130131	total: 27.5ms	remaining: 7.73s
5:	learn: 0.3068145	total: 32.6ms	remaining: 7.63s
6:	learn: 0.3008564	total: 37.5ms	remaining: 7.52s
7:	learn: 0.2957078	total: 42.6ms	remaining: 7.46s
8:	learn: 0.2912627	total: 47.1ms	remaining: 7.33s
9:	learn: 0.2869100	total: 51.6ms	remaining: 7.22s
10:	learn: 0.2830463	total: 56.5ms	remaining: 7.19s
11:	learn: 0.2796712	total: 61.1ms	remaining: 7.11s
12:	learn: 0.2771517	total: 65.6ms	remaining: 7.05s
13:	learn: 0.2736100	total: 70.3ms	remaining: 7.01s
14:	learn: 0.2714338	total: 75ms	remaining: 6.98s
15:	learn: 0.2685333	total: 79.6ms	remaining: 6.94s
16:	learn: 0.2659293	total: 84.3ms	remaining: 6.91s
17:	learn: 0.2640862	total: 89.3ms	remaining: 6.9s
18:	learn: 0.2618646	total: 93.9ms	remaining: 6.88s
19:	learn: 0.2604424	total: 

In [23]:
ensemble_results_df

,MSE,MAE,RMSE,R2 Score
0,0.04185,0.155396,0.204573,0.667357


The ensemble model combining CatBoost, XGBoost, and LightGBM achieved the best overall performance, with an MSE of 0.04185, MAE of 0.155396, RMSE of 0.204573, and an R² score of 0.667357.
